# import modules

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline

# Data set 불러오기

In [2]:
X_df=pd.read_excel('../20200804 bigcon/Train set.xlsx',header = 1)

In [29]:
# 데이터 reset 하고싶으면 위의 셀 말고 이거 돌려가면서 reset 하세요! 그게 제일 빨라요
X = X_df.copy()

In [30]:
X.head() 

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0


In [31]:
X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2,2019-01-01 06:20:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
4,2019-01-01 06:40:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0
...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN


# 곁측치 처리

In [32]:
X.isnull().sum()

방송일시         0
노출(분)    16784
마더코드         0
상품코드         0
상품명          0
상품군          0
판매단가         0
취급액        937
dtype: int64

## 상품군 곁측치 처리

 판매단가가 0 인 상품은 예측에서 제외

In [33]:
def NA_goods(df=X):
    df=df.loc[df['판매단가'] != 0,:]
    df = df.reset_index(drop=True)
    return df

In [34]:
NA_goods(X)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0
...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,NaN,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0
37369,2020-01-01 00:00:00,NaN,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0
37370,2020-01-01 00:00:00,NaN,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0


## 노출(분) 곁측치 처리

노출분은 같은 시간에 같이 방송되어서 NaN 으로 뜨는것이다. <br>
아마 같은 시간에 방송되어진 값을 평균해서 채우면 될듯?

In [35]:
# 0 노출시간이 0 인 데이터가 없다.
# 즉 Na 처리할떄 0으로 대채한 후에 평균떄려도 아무 상관 없다는 이야기!
(X.loc[:,'노출(분)']==0).sum()

0

In [36]:
# 노출(분) 을 제일 먼저 시작한 시간의 평균으로 만드는 FUNCTION 정의
# 아직 맞는지는 잘 몰라서 안맞는 경우 수정해주세요.
def NA_minute(df=X):
    # 곁측치를 0으로 대체
    X['노출(분)'] = X['노출(분)'].fillna(0)
    
    count_0 = [] 
    count_not0 = [] # 0이 아닌 수의 index
    for idx in df.index :
        if df.loc[idx,'노출(분)'] == 0 :
            count_0.append(idx)
        else :
            count_not0.append(idx)
    count_not0.append(len(df)) # 맨 마지막 열에 0 인 경우를 대비하여

    for idx,x in enumerate(count_not0):
        if idx != len(count_not0)-1: 
            if (count_not0[idx+1] - count_not0[idx] - 1) != 0:
                start = count_not0[idx] 
                end = count_not0[idx+1]
                num = end - start 
                df.loc[start:(end-1),'노출(분)'] = df.loc[start,'노출(분)'] / num 
    return df

In [37]:
NA_minute(X)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2019-01-01 06:00:00,10.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
1,2019-01-01 06:00:00,10.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2,2019-01-01 06:20:00,10.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
3,2019-01-01 06:20:00,10.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
4,2019-01-01 06:40:00,10.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0
...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN


## 곁측치 처리 pipeline

In [38]:
def Na_pipeline(df = X):
    df = NA_goods(df)
    df = NA_minute(df)
    return(df)

In [39]:
X = Na_pipeline(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2019-01-01 06:00:00,10.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
1,2019-01-01 06:00:00,10.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2,2019-01-01 06:20:00,10.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
3,2019-01-01 06:20:00,10.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
4,2019-01-01 06:40:00,10.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0
...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,5.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0
37368,2020-01-01 00:00:00,5.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0
37369,2020-01-01 00:00:00,5.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0
37370,2020-01-01 00:00:00,5.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0


In [40]:
# na 없다
X.isnull().sum().sum()

0

# 데이터 살펴보기

노출 분은 대부분 10 분 내외인듯.

In [41]:
pd.DataFrame(pd.Series(X['노출(분)']).value_counts(normalize=True))

,노출(분)
20.000000,0.299502
10.000000,0.225222
5.000000,0.157284
3.333333,0.096329
6.666667,0.074039
...,...
11.033333,0.000027
13.500000,0.000027
17.083333,0.000027
19.150000,0.000027


# 변수 늘리기

## 시간 데이터 늘리기

In [97]:
# 시각은 우선 시,분을 합쳐서 하나의 값으로 만들었습니다.
def time_feature(df = X):
    df['방송일시']=df['방송일시'].astype('str')
    df['날짜']=df['방송일시'].str.split(' ').str[0]
    df['시간']=df['방송일시'].str.split(' ').str[1]
    df['년']=df['날짜'].str.split('-').str[0]
    df['월']=df['날짜'].str.split('-').str[1]
    df['일']=df['날짜'].str.split('-').str[2]
    df['시']=df['시간'].str.split(':').str[0]
    df['분']=df['시간'].str.split(':').str[1]
    df['초']=df['시간'].str.split(':').str[2]
    df['시각'] = df['시'].astype('float32') + df['분'].astype('float32')*(1/60)
    df.drop(['시','분','초','시간','날짜'],axis=1,inplace = True)
    return(df)

In [99]:
X = time_feature(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,년,월,일,시각
0,2019-01-01 06:00:00,10.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,2019,01,01,6.000000
1,2019-01-01 06:00:00,10.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,2019,01,01,6.000000
2,2019-01-01 06:20:00,10.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,2019,01,01,6.333333
3,2019-01-01 06:20:00,10.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,2019,01,01,6.333333
4,2019-01-01 06:40:00,10.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,2019,01,01,6.666667
...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,5.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,2019,12,31,23.666667
37368,2020-01-01 00:00:00,5.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,2020,01,01,0.000000
37369,2020-01-01 00:00:00,5.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,2020,01,01,0.000000
37370,2020-01-01 00:00:00,5.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,2020,01,01,0.000000


## 식사 시간과 관련된 변수

식사시간의 경우 유의미한 데이터가 있을 수 있다. 그 시간대를 잡아서 변수를 늘리면 효과가 있을거같다. <br>
하지만 아침 식사시간의 경우 거르는 인구가 33%(통계청) 이나 되는 이유로, 오히려 혼란을 가중시킬 수 있기떄문에 그냥 점심과 저녁만 고려하자

In [109]:
def eat_feature(df = X):
    df['점심'] = 0
    df.loc[ (11.5<df['시각']) & (df['시각'] < 13),'점심']  = 1
    df['저녁'] = 0
    df.loc[ (17.5<df['시각'])&(df['시각']<19.5) , '저녁'] = 1
    return(df)

In [110]:
X = eat_feature(X)

In [111]:
X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,년,월,일,시각,점심,저녁
0,2019-01-01 06:00:00,10.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,2019,01,01,6.000000,0,0
1,2019-01-01 06:00:00,10.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,2019,01,01,6.000000,0,0
2,2019-01-01 06:20:00,10.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,2019,01,01,6.333333,0,0
3,2019-01-01 06:20:00,10.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,2019,01,01,6.333333,0,0
4,2019-01-01 06:40:00,10.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,2019,01,01,6.666667,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,5.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,2019,12,31,23.666667,0,0
37368,2020-01-01 00:00:00,5.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,2020,01,01,0.000000,0,0
37369,2020-01-01 00:00:00,5.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,2020,01,01,0.000000,0,0
37370,2020-01-01 00:00:00,5.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,2020,01,01,0.000000,0,0


In [112]:
pd.Series(df['점심']).value_counts() # 좀 sparse 하긴 한데 의미는 있을거라고 믿는다...

0    35025
1     2347
Name: 점심, dtype: int64

## Feature 늘리기 Pipeline

# 데이터 type 처리

상품군에 대해서 onehot encoding 적용하자

아 그런데 시각 데이터에 대해서, 0 과 24에 가까운 값들은 비슷한 속성을 가짐에도(늦은시간) 값이 차이가 많이난다.
그러므로 내 생각에는 그냥 categorical 로 싹다 해버리는게 좋을거같은데..?